In [146]:
import geopandas as gpd
from folium.plugins import MarkerCluster


# 读取 GeoJSON 文件
gdf = gpd.read_file('phidu_premature_mortality_by_cause_phn_2014_18-.geojson')


tolerance = 0.001
gdf['geometry'] = gdf['geometry'].simplify(tolerance=tolerance, preserve_topology=True)



In [147]:
gdf['dths_rspr1_formatted'] = gdf['dths_rspr1'].apply(lambda x: f"{x:.2f}")

In [148]:
import folium
m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=12)



/tmp/ipykernel_27569/1017310664.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=12)


In [149]:
def calculate_clusters(gdf, grid_size):
    clusters = []
    for _, row in gdf.iterrows():
        centroid = row['geometry'].centroid
        x = int(centroid.x / grid_size)
        y = int(centroid.y / grid_size)
        cluster = next((c for c in clusters if c['x'] == x and c['y'] == y), None)
        if cluster:
            cluster['dths_rspr1_sum'] += row['dths_rspr1']
            cluster['points'].append(row['geometry'])
        else:
            clusters.append({
                'x': x, 
                'y': y, 
                'dths_rspr1_sum': row['dths_rspr1'], 
                'centroid': Point(centroid.x * grid_size, centroid.y * grid_size),
                'points': [row['geometry']]
            })
    return clusters

In [153]:
clusters = calculate_clusters(gdf, m.get_zoom())

AttributeError: 'Map' object has no attribute 'get_zoom'

In [ ]:
for cluster in clusters:
    folium.Marker(
        location=[cluster['centroid'].y, cluster['centroid'].x],
        icon=folium.DivIcon(html=f"""<div style="font-family: sans-serif; color: black; font-size: 12pt">{cluster['dths_rspr1_sum']:.2f}</div>""")
    ).add_to(m)

In [ ]:
m.save('map.html')

In [ ]:
m